In [1]:
from code.neuron import *
from code.encodingsource import *
from code.hsgs import *
from code.classical_neuron import *
from code.classical_pso import *
from code.sf import *
simulator = Aer.get_backend('qasm_simulator')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

from code.experiment_functions import *

In [ ]:
diabetes = pd.read_csv('data/diabetes.csv')

In [ ]:
# normalization 0-1
for i in diabetes.columns:
    diabetes[i] = (diabetes[i] - diabetes[i].min()) / (diabetes[i].max() - diabetes[i].min())

In [ ]:
drop_indices = np.random.choice(diabetes.index, 368, replace=False)
diabetes = diabetes.drop(drop_indices)

In [ ]:
# split in test and train data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(diabetes.drop('Outcome',axis=1), diabetes['Outcome'], test_size=0.2)

In [ ]:
X_train[X_train == 0] = 0.000001
X_test[X_test == 0] = 0.000001

In [ ]:
X_train = X_train.values.tolist()
X_test = X_test.values.tolist()
y_train = y_train.values.tolist()
y_test = y_test.values.tolist()

In [ ]:
y_train = [int(x) for x in y_train]
y_test =  [int(x) for x in y_test]

In [ ]:
"""
with open('results/version4/train_data.json', 'w+') as fp:
    json.dump([X_train, y_train], fp)
    
with open('results/version4/test_data.json', 'w+') as fp:
    json.dump([X_test, y_test], fp)
"""

In [ ]:
with open('results/version4/train_data.json') as json_file:
    X_train = json.load(json_file)[0]

with open('results/version4/train_data.json') as json_file:
    y_train = json.load(json_file)[1]
    
with open('results/version4/test_data.json') as json_file:
    X_test = json.load(json_file)[0]

with open('results/version4/test_data.json') as json_file:
    y_test = json.load(json_file)[1]

In [ ]:
def float_to_binary(list_, m, n):
    """Convert the float value `x` to a binary string of length `m + n`
    where the first `m` binary digits are the integer part and the last
    'n' binary digits are the fractional part of `x`.
    """
    list_scaled = []
    for number in list_:
        x_scaled = round(number * 2 ** n)
        x_scaled = '{:0{}b}'.format(x_scaled, m + n)
        for i in x_scaled:
            if i == '0':
                list_scaled.append(-1)
            else:
                list_scaled.append(1)
    return list_scaled

X_test_bin = []
for i in X_test:
    X_test_bin.append(float_to_binary(i, 1, 4))
    
X_train_bin = []
for i in X_train:
    X_train_bin.append(float_to_binary(i, 1, 4))

## Test

In [ ]:
w = list(np.random.uniform(low=0.0, high=1.0, size=(len(X_test_bin[0]),)))

In [ ]:
weights = quantumNeuronFIT(X_test_bin, 
                           y_train,
                           init_weight = w,
                           threshold=0.7,
                           lrParameter=0.1, 
                           n_epochs=50, 
                           trainingBias=False,
                           epoch_results=True,
                           phaseEstrategyOperator='original',
                           trainingApproaches={'hsgs'})

In [ ]:
results = quantumNeuronPREDICT(X_test, 
                                   y_test,
                                   threshold=0.7,
                                   weightVectorsPhaseEncoding = weights[0], 
                                   weightVectorsHSGS = weights[1], 
                                   repeat=5,
                                   bias=False, 
                                   phaseEstrategyOperator='original',
                                   testingApproaches={'hsgs'})

In [ ]:
print('phase avg_error', results[0])
print('phase weights', results[2])

print('\HSGS avg_error', results[3])
print('HSGS weights', results[5])

## Binary Quantum Neuron Experiments 

In [ ]:
output = {'model':[],
          'phase_strategy':[],
            'bias':[],
            'threshold':[],
            'lr':[],
            'avg_error':[],
            'trained_weights':[],
            'initial_weights':[],
            'neuron_outputs':[]}

In [ ]:
# search space
initial_weights =  list(np.random.uniform(low=0.0, high=1.0, size=(len(X_train_bin[0]),)))
threshold_space = [0.5, 0.7, 0.9]
lr_space = [0.1]
bias_space = [False, True]
strategy_space = ['original']

In [ ]:
%%time
for w in initial_weights:
    for strategy in strategy_space:
        for threshold in threshold_space:
            for lr in lr_space:
                for bias in bias_space:
                    # execute experiment
                    weights = quantumNeuronFIT(X_train_bin, 
                                               y_train,
                                               init_weight=initial_weights,
                                               threshold=threshold,
                                               lrParameter=lr, 
                                               n_epochs=50, 
                                               trainingBias=bias,
                                               phaseEstrategyOperator=strategy,
                                               trainingApproaches={'hsgs'})

                    results = quantumNeuronPREDICT(X_test_bin, 
                                                   y_test,
                                                   threshold=threshold,
                                                   weightVectorsPhaseEncoding = weights[0], 
                                                   weightVectorsHSGS = weights[1], 
                                                   repeat=5,
                                                   bias=bias, 
                                                   phaseEstrategyOperator=strategy,
                                                   testingApproaches={'hsgs'})


                    output['model'].append('HSGS')
                    output['lr'].append(lr)
                    output['bias'].append(bias)
                    output['threshold'].append(threshold)
                    output['phase_strategy'].append(strategy)
                    output['avg_error'].append(results[3])
                    output['neuron_outputs'].append(results[4])
                    output['trained_weights'].append(results[5])
                    output['initial_weights'].append(w)
                    print('\n')

In [ ]:
experiment_hsgs = pd.DataFrame(output)

## Continuously Valued Quantum Neuron Experiments

In [ ]:
output = {'model':[],
          'phase_strategy':[],
            'bias':[],
            'threshold':[],
            'lr':[],
            'avg_error':[],
            'trained_weights':[],
            'initial_weights':[],
            'neuron_outputs':[]}

In [ ]:
# search space
initial_weights =  list(np.random.uniform(low=0.0, high=1.0, size=(len(X_test[0]),)))
threshold_space = [0.5, 0.7, 0.8]
lr_space = [0.1]
bias_space = [False, True]
strategy_space = ['angleradius']

In [ ]:
%%time
for w in initial_weights:
    for strategy in strategy_space:
        for threshold in threshold_space:
            for lr in lr_space:
                for bias in bias_space:
                    # execute experiment
                    weights = quantumNeuronFIT(X_train, 
                                               y_train,
                                               init_weight=initial_weights,
                                               threshold=threshold,
                                               lrParameter=lr, 
                                               n_epochs=50, 
                                               trainingBias=bias,
                                               phaseEstrategyOperator=strategy,
                                               trainingApproaches={'phase-encoding'})

                    results = quantumNeuronPREDICT(X_test, 
                                                       y_test,
                                                       threshold=threshold,
                                                       weightVectorsPhaseEncoding = weights[0], 
                                                       weightVectorsHSGS = weights[1], 
                                                       repeat=5,
                                                       bias=bias, 
                                                       phaseEstrategyOperator=strategy,
                                                       testingApproaches={'phase-encoding'})

                    # get experiment results
                    output['model'].append('phase-encoding')
                    output['lr'].append(lr)
                    output['bias'].append(bias)
                    output['threshold'].append(threshold)
                    output['phase_strategy'].append(strategy)
                    output['avg_error'].append(results[0])
                    output['neuron_outputs'].append(results[1])
                    output['trained_weights'].append(results[2])
                    output['initial_weights'].append(w)
                    print('\n')

In [ ]:
experiment_phase = pd.DataFrame(output)

### Save results

In [ ]:
experiments = pd.concat([experiment_hsgs, experiment_phase])

In [ ]:
experiments.to_csv('data_and_results/diabetes/experiments_diabetes.csv')